In [ ]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
import geopandas as gp
import os

%matplotlib inline
plt.style.use('ggplot')


In [ ]:
base_dir = "../../../../Outcome Measurement Data/MALARIA/"
# Esto no incluye toda las áreas del país. Sólo las áreas vistas en la base de datos de notifs.
notifs_das2depto = {
    "Escuintla": 5,
    "Alta Verapaz": 16,
    "Izabal" : 18,
    "Suchitepéquez" : 10,
    "Santa Rosa" : 6,
    "Retalhuleu" : 11,
    "Petén Norte" : 17,
    "Petén SurOccidental" : 17,
    "Huehuetenango" : 13,
    "Quiché" : 14,
    "Petén Sur Oriental" : 17,
    "Baja Verapaz" : 15,
    "Sololá" : 7,
    "Zacapa" : 18,
    "Ixcán" : 14,
    "San Marcos" : 12,
    "Chimaltenango" : 4,
    "Guatemala Central" : 1,
    "Guatemala Sur" : 1,
    "Ixil" : 14,
    "Jutiapa" : 22,
    "Chiquimula" : 20,
    "Ixcan" : 14,
    "El Progreso" : 2,
    "Guatemala Nororiental" : 1,
    "Quetzaltenango" : 9,
}

In [ ]:
deptos = gp.read_file("../../../../Covariates and Other Data/GIS/GT-IGN-cartografia_basica-Departamentos.geojson")
deptos = deptos[lambda x: x.CODIGO.astype(int) <= 2200]

In [ ]:
bednets = pd.read_csv("../../../../Outcome Measurement Data/MALARIA/GTM - Bednets Distr_1.tar.gz", index_col = 0 )

In [ ]:
notifs = pd.read_excel(base_dir + "Base de datos 2016 - 2018 NAD Consolidado 04.06.xlsx")


In [ ]:
bednetsYears = bednets.groupby(["year", "CodDepto_e"]).Pabellones.sum().unstack(0).fillna(0)
bednetsYears

In [ ]:
notifs["DAS"] = notifs.AREA.map(str.strip).map(notifs_das2depto.get)

In [ ]:
notifsYear = notifs.groupby(["Año", "DAS"]).AREA.count().unstack(0).fillna(0)
notifsYear

In [ ]:
def plotNotifsVsBednets(ax, year = 2015):
    temp = pd.DataFrame(data = [bednetsYears[year], notifsYear[year]]).T.fillna(0)
    temp = np.log10(temp)
    corr = np.round(temp.corr().values[0][1], 2)
    temp.columns = ["Bednets (Log)", "Notifications (Log)"]
    temp.plot.scatter("Bednets (Log)", "Notifications (Log)", 
                      title="Benets distributed vs Malaria Notifications\nYear "+ str(year),
                          ax = ax)
    plt.text(0.05,0.9, "Correlation = "+str(corr), transform=ax.transAxes)

mlp.rcParams['figure.figsize'] = [17.0, 4.0]
fig, axes = plt.subplots(1,3)
plotNotifsVsBednets(axes[0], 2015)
plotNotifsVsBednets(axes[1], 2016)
plotNotifsVsBednets(axes[2], 2017)


In [ ]:
pblYears = bednetsYears.divide(1000).reset_index()\
                         .assign(deptocode=lambda x: (x.CodDepto_e * 100).astype(int).astype(str)\
                                 .map(lambda y: y if y != 1 else "101"))

In [ ]:
vmax = pblYears[list(range(2012,2018))].max().max()

side = 8
n = 7
vmin = 0
#vmax = 8
f, axes = plt.subplots(int(np.ceil(n/2)), 2, figsize=(side*2*1.1, side*np.ceil(n/2) ))
i = 2012
for row in axes:
    for ax in row:
        if i not in pblYears:
            ax.axis("off")
            continue
        temp3 = deptos.merge(pblYears[[i, "deptocode"]], 
                             left_on="CODIGO", right_on = "deptocode", how="left" ).fillna(0)
        temp3[temp3[i] == 0].plot(ax=ax, color="grey")
        if sum(temp3[i] > 0) > 0:
            temp3[temp3[i] > 0].plot(i, ax = ax, cmap = "Blues", 
                                             legend=True, edgecolor="black",
                                             vmin = vmin, vmax = vmax)
        ax.axis('off')
        ax.set_title("Malaria bednets delivered\nThousands by department for "+ str(i))
        i += 1